In [1]:
from transformers import AutoTokenizer, GPTNeoXForCausalLM

checkpoint = "OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = GPTNeoXForCausalLM.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
inputs = tokenizer("<|prompter|>What is a meme, and what's the history behind this word?<|endoftext|><|assistant|>", return_tensors="pt")
tokens = model.generate(**inputs, do_sample=True, temperature=0.9, max_length=100)
tokenizer.decode(tokens[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'What is a meme, and what\'s the history behind this word?A meme is a cultural idea, behavior, or style that spreads from person to person within a population. The word "meme" was first used by Richard Dawkins in The Selfish Gene in 1976 to describe units of cultural information that replicate and spread, such as jokes, catchphrases, and rituals. The term quickly became popular and is now used to describe a wide range of cultural phenomena'